---
title: Tracking Changes
date: 2023-11-30 
authors:
  - name: Sébastien Boisgérault
    email: Sebastien.Boisgerault@minesparis.psl.eu
    url: https://github.com/boisgera
    affiliations:
      - institution: Mines Paris - PSL University
        department: Institut des Transformation Numériques (ITN)
github: boisgera
license: CC-BY-4.0
open_access: true
---

In order to understand how `.tldr` files are structured, we can add a new graphical objects, change some if their properties, etc. and each time we modify the document, analyze the corresponding evolution of the file.

In this notebook, we develop some tooling to help us track such changes.

## Text comparison

We define two similar versions of the "zen of Python":

In [1]:
zen_1 = """The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Errors should never pass silently.
In the face of ambiguity, refuse the temptation to guess.
There should be one obvious way to do it.
Although that way may not be obvious at first.
Now is better than never.
Although never is often better than right now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it is a good idea.
"""

In [2]:
zen_2 = """\
The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
"""

```{exercise}
 1. Transform `zen_1` and `zen_2` into list of lines.
 2. Use the [`difflib`](https://docs.python.org/3/library/difflib.html) module of the Python standard library to [`compare`](https://docs.python.org/3/library/difflib.html#difflib.Differ.compare) the two sequences.
 3. Make a text out of the output of compare and print it.
 4. Interpret the result and list the differences between both versions of the zen of Python.
```

**Exercice 1**

1.

In [3]:
zen_1_list = zen_1.splitlines()
zen_2_list = zen_2.splitlines()

2. et 3.

In [4]:
import difflib
diff = difflib.unified_diff(zen_1_list, zen_2_list, lineterm='')
for line in diff:
    print(line)

--- 
+++ 
@@ -8,11 +8,14 @@
 Sparse is better than dense.
 Readability counts.
 Special cases aren't special enough to break the rules.
+Although practicality beats purity.
 Errors should never pass silently.
+Unless explicitly silenced.
 In the face of ambiguity, refuse the temptation to guess.
-There should be one obvious way to do it.
-Although that way may not be obvious at first.
+There should be one-- and preferably only one --obvious way to do it.
+Although that way may not be obvious at first unless you're Dutch.
 Now is better than never.
-Although never is often better than right now.
+Although never is often better than *right* now.
 If the implementation is hard to explain, it's a bad idea.
-If the implementation is easy to explain, it is a good idea.
+If the implementation is easy to explain, it may be a good idea.
+Namespaces are one honking great idea -- let's do more of those!


4. Les différences entre les textes se situent aux lignes où l'on retrouve les + et - au début. Les - concernent les lignes différentes sur zen_1 et les + celles sur zen_2. Si on a une seule ligne au mileu des autres qui est repérée, c'est qu'elle est un pur rajout ou manque. Si on voit plusieurs lignes à la suite avec un + et un -, c'est surement qu'il y a une différence d'orthographe ou de syntaxe entre les lignes. 

Ainsi, on retrouve trois lignes dans zen_2 qui ne sont pas dans zen_1 :

    - Although practicality beats purity.

    - Unless explicitly silenced

    - Namespaces are one honking great idea -- let's do more of those!

On a des mots rajoutés dans zen_2:

    - -- and preferably only one --

    - unless you're Dutch.

Dans zen_2 il y a aussi des étoiles rajoutées autour de right dans "right now"

Enfin, dans zen_2 il est dit non pas "it is a good idea" mais "it may be a good idea"


We can make our job easier if we use HTML instead of plain text to visualise the differences between the two texts.


```{exercise}
  1. Use the [HtmlDiff](https://docs.python.org/3/library/difflib.html#difflib.HtmlDiff) class of difflib to produce a `diff.html` file that represents this difference in a HTML document.
  2. Use the [webbrowser](https://docs.python.org/3/library/webbrowser.html) module of the standard library to open it!
  3. Define a `display_diff_text` function that takes two arguments `text_1` and `text_2` and automates steps 1. and 2.
```

**Exercice 2**

1.

In [5]:
diff_html = difflib.HtmlDiff().make_file(zen_1.splitlines(), zen_2.splitlines(), fromdesc='zen_1', todesc='zen_2')

with open("C:/Users/doria/cours_info/langage/tldraw-project/docs/diff.html", "w") as file:
    file.write(diff_html)

2.

In [6]:
import webbrowser 

webbrowser.open("C:/Users/doria/cours_info/langage/tldraw-project/docs/diff.html")

True

3.

In [7]:
def display_diff_text(text_1, text_2):
    import webbrowser
    import difflib
    diff_html = difflib.HtmlDiff().make_file(text_1.splitlines(), text_2.splitlines(), fromdesc='text_1', todesc='text_2')
    with open("diff_text.html", "w") as file:
        file.write(diff_html)
    webbrowser.open("diff_text.html")

## Comparison of JSON documents

````{exercise} Comparison of dictionnaries

 1. Create a `display_diff` function that takes two Python objects, converts them to strings then leverages `display_diff_text` to display the difference in a browser.

 2. Consider the 3 dictionaries defined by
    ```python
    d1 = {k:k+1 for k in range(100)}
    d2 = d1.copy(); d2[50] = 50
    d3 = {k:k+1 for k in range(99, -1, -1)}
    ```
    `d1` and `d2` have a slight difference and `d1` and `d2` are equal.
    Does your `display_diff` function make easy to spot where the difference is in the first case when it compares `d1` and `d2`?
    Does it make easy to see that `d1` and `d3` are equal?

  3. Investigate the [`pprint`](https://docs.python.org/3/library/pprint.html) module standard library ; use it to improve the behavior of `display_text_diff` in the two cases considered in the previous question.

````
 

**Exercice 3**

1.

In [8]:
def display_diff(obj_1, obj_2):
    str_obj_1 = str(obj_1)
    str_obj_2 = str(obj_2)
    display_diff_text(str_obj_1, str_obj_2)

2.

In [9]:
d1 = {k:k+1 for k in range(100)}
d2 = d1.copy(); d2[50] = 50
d3 = {k:k+1 for k in range(99, -1, -1)}

In [10]:
display_diff(d1,d2)

In [11]:
display_diff(d1,d3)

La fonction `display_diff` ne rend pas les différences et égalitées entre les listes évidentes, que ce soit entre `d1 et d2` ou entre `d1 et d3`. 



3.

In [12]:
def display_diff_text(text_1, text_2):
    import pprint
    text_1_pprint = pprint.pformat(text_1)
    text_2_pprint = pprint.pformat(text_2)
    
    diff_html = difflib.HtmlDiff().make_file(text_1_pprint.splitlines(), text_2_pprint.splitlines(), fromdesc='text_1', todesc='text_2')
    with open("C:/Users/doria/cours_info/langage/tldraw-project/docs/diff_text.html", "w") as file:
        file.write(diff_html)
    webbrowser.open("C:/Users/doria/cours_info/langage/tldraw-project/docs/diff_text.html")

In [13]:
display_diff(d1,d2)

In [14]:
display_diff(d1,d3)

La fonction permet maintenant de bien mettre en évidence la différence entre `d1 et d2`, cependant, du fait que `d1 et d3` sont identique mais en sens inverse, le programme ne voit pas qu'elles sont identiques. 

```{exercise} tldraw documents comparator
Implement a function `tldraw_diff` that takes as argument two filenames that refer to tldraw documents and display their differences in the browser.
```

**Exercice 4**

In [15]:
def read_tldraw_file(filename):
    with open(filename, "r") as file:
        tldraw = file.read()
    return tldraw

def tldraw_diff(file1, file2):
    tldraw1 = read_tldraw_file(file1)
    tldraw2 = read_tldraw_file(file2)
    display_diff_text(tldraw1, tldraw2)